## Imports

In [45]:
import pmp_functions_v5 as pmp
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

## Data

### Strategies

In [46]:
# --- Fundamental Data ---
fundamental_equity = pd.read_csv("../Results/fundamental_equity.csv", index_col=0, parse_dates=True)
fundamental_equity.rename(columns={"w_GB": "w_UK"}, inplace=True)

# --- Macro Data ---
macroRS_equity = pd.read_csv("../Results/macroRS_equity.csv", index_col=0, parse_dates=True)

# --- Trend Following Data ---
trend_equity = pd.read_csv("../Results/trend_equity.csv", index_col=0, parse_dates=True)
trend_bond = pd.read_csv("../Results/trend_bond.csv", index_col=0, parse_dates=True)
trend_rates = pd.read_csv("../Results/trend_rates.csv", index_col=0, parse_dates=True)
trend_fx = pd.read_csv("../Results/trend_fx.csv", index_col=0, parse_dates=True)

In [47]:
def extract_clean_weights(df):
    # 1) Nur die weight-Spalten nehmen
    weight_cols = [c for c in df.columns if c.startswith("w_")]
    
    # 2) Gewichtstabelle extrahieren
    w = df[weight_cols].copy()
    
    # 3) Prefix "w_" entfernen
    w.columns = [c.replace("w_", "") for c in weight_cols]
    
    return w

# Clean weights for both strategies
fundamental_equity = extract_clean_weights(fundamental_equity)

trend_equity = extract_clean_weights(trend_equity)
trend_bond = extract_clean_weights(trend_bond)
trend_rates = extract_clean_weights(trend_rates)
trend_fx = extract_clean_weights(trend_fx)

macroRS_weights = extract_clean_weights(macroRS_equity)


### Asset Data

In [48]:
# --- Load Equity Price Data ---
equity_prices = pd.read_excel(
    "../Data_Ryan/Equity Data.xlsx",
    index_col = 0,
    parse_dates = True
)
equity_prices.index = pd.to_datetime(equity_prices.index)
equity_prices.index = equity_prices.index + pd.offsets.MonthEnd(0)

equity_returns = equity_prices.pct_change()
equity_returns

,US,AU,CH,JP,UK,EM,EU
Date,,,,,,,
1997-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-10-31,-0.031954,NaN,NaN,-0.069246,-0.053835,NaN,NaN
1997-11-30,0.033550,NaN,NaN,-0.071232,0.009611,NaN,NaN
1997-12-31,0.013571,NaN,NaN,-0.075596,0.029754,NaN,NaN
1998-01-31,0.008938,NaN,NaN,0.085984,0.044176,NaN,NaN
...,...,...,...,...,...,...,...
2025-06-30,0.047850,0.034649,0.008742,0.018397,0.017408,0.087457,0.024923
2025-07-31,0.019268,-0.002424,-0.029467,-0.011530,0.000891,0.009281,-0.026534
2025-08-31,0.015453,0.045088,0.049595,0.068686,0.032134,0.008553,0.028645


In [49]:
# --- Load Bond Futures ---
bond_futures = pd.read_excel(
    "../Data_Ryan/Bond Data.xlsx",
    index_col = 0,
    parse_dates = True
)
bond_futures.index = pd.to_datetime(bond_futures.index)
bond_futures.index = bond_futures.index + pd.offsets.MonthEnd(0)

bond_returns = bond_futures.pct_change()
bond_returns

,EU,JP,AU,US,CH,EM,UK
Dates,,,,,,,
1998-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-28,0.017379,0.006342,-0.000532,-0.009009,0.013870,NaN,-0.000851
1998-03-31,-0.014688,0.003317,0.002553,-0.002674,-0.009084,NaN,0.039250
1998-04-30,0.026085,0.008926,-0.001061,-0.000536,-0.013143,NaN,0.002005
1998-05-31,0.015326,0.017256,0.005684,0.005901,0.021489,NaN,-0.013159
...,...,...,...,...,...,...,...
2025-07-31,-0.032262,-0.007101,-0.022422,-0.009475,-0.015542,-0.009048,-0.044820
2025-08-31,0.023240,-0.003576,0.017618,0.012519,0.020001,0.005948,0.003818
2025-09-30,0.004901,-0.005493,0.010887,0.000139,0.021848,0.001495,0.000005


In [50]:
# --- Load Rates Futures ---
rates_futures = pd.read_excel(
    "../Data_Ryan/Interest Rates Data.xlsx",
    index_col = 0,
    parse_dates = True
)
rates_futures.index = pd.to_datetime(rates_futures.index)
rates_futures.index = rates_futures.index + pd.offsets.MonthEnd(0)

rates_returns = rates_futures.pct_change()
rates_returns

,CH,EU,AU,US,EM,UK,JP
Date,,,,,,,
1998-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-03-31,-0.002432,NaN,0.000844,0.000524,0.013047,NaN,0.001312
1998-04-30,-0.000509,NaN,-0.000730,-0.000209,0.013644,NaN,0.000302
1998-05-31,0.000000,NaN,0.000422,-0.000733,-0.014970,NaN,0.000554
...,...,...,...,...,...,...,...
2025-07-31,0.000000,-0.001019,-0.002281,0.000000,0.000000,-0.001840,0.000000
2025-08-31,0.000000,-0.000459,-0.000207,0.000000,-0.009700,-0.002765,0.000000
2025-09-30,0.000000,0.000051,-0.001759,0.000000,-0.004021,-0.001232,0.000000


In [51]:
# --- Load Currency Prices ---
fx_data = pd.read_excel(
    "../Data_Ryan/FX Data.xlsx",
    sheet_name = 'RETURNS',
    index_col = 0,
    parse_dates = True
)
fx_data.index = pd.to_datetime(fx_data.index)
fx_data.index = fx_data.index + pd.offsets.MonthEnd(0)
fx_returns = fx_data
fx_returns



,CH,EU,JP,UK,AU,EM
Date,,,,,,
1989-01-31,-0.062505,NaN,-0.044448,-0.028124,0.043550,NaN
1989-02-28,0.020885,NaN,0.025663,-0.002575,-0.095372,NaN
1989-03-31,-0.067754,NaN,-0.048472,-0.030821,0.027844,NaN
1989-04-30,-0.009532,NaN,-0.006180,0.004816,-0.025641,NaN
1989-05-31,-0.022300,NaN,-0.070709,-0.068691,-0.047106,NaN
...,...,...,...,...,...,...
2025-06-30,0.033185,0.036774,-0.003389,0.020141,0.022884,0.004021
2025-07-31,-0.027679,-0.033666,-0.048025,-0.038372,-0.024318,-0.009627
2025-08-31,0.010759,0.021636,0.021495,0.022178,0.017316,0.009796


### Riskfree Data

In [52]:
# --- Load Riskfree Rate ---
factors_data = pd.read_excel(
    "../Data_Ryan/Factors.xlsx",
    index_col = 0,
    parse_dates = True
)

factors_data.index = pd.to_datetime(factors_data.index, format='%Y%m')
factors_data.index = factors_data.index + pd.offsets.MonthEnd(0)
factors_data /= 100

riskfree = factors_data["RF"].resample('ME').last()
riskfree

/var/folders/37/zcyd0byn17lfrmxtt4_4k3s40000gn/T/ipykernel_90111/2112351923.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  factors_data = pd.read_excel(


1926-07-31    0.0022
1926-08-31    0.0025
1926-09-30    0.0023
1926-10-31    0.0032
1926-11-30    0.0031
               ...  
2025-06-30    0.0034
2025-07-31    0.0034
2025-08-31    0.0038
2025-09-30    0.0033
2025-10-31    0.0037
Freq: ME, Name: RF, Length: 1192, dtype: float64

### Benchmark Data

In [53]:
# --- Benchmark Data ---
benchmark_data = pd.read_excel(
    "../Data_Ryan/Benchmarks.xlsx",
    index_col = 0,
    parse_dates = True
)

benchmark_data.index = pd.to_datetime(benchmark_data.index)
benchmark_data = benchmark_data.resample('ME').last()


benchmark_return = benchmark_data[['MSCI World']].pct_change()
benchmark_return = benchmark_return.squeeze()
benchmark_return

Date
1986-12-31         NaN
1987-01-31         NaN
1987-02-28         NaN
1987-03-31         NaN
1987-04-30         NaN
                ...   
2025-07-31    0.013121
2025-08-31    0.026408
2025-09-30    0.032574
2025-10-31    0.020226
2025-11-30    0.003149
Freq: ME, Name: MSCI World, Length: 468, dtype: float64

### Factor Data

In [54]:
# --- Load Factors Data ---
famafrench_data = pd.read_csv(
    "../Data_Ryan/famafrench_factors.csv",
    index_col = 0,
    parse_dates = True
)

famafrench_data.index = pd.to_datetime(famafrench_data.index, format='%Y%m')
famafrench_data.index = famafrench_data.index + pd.offsets.MonthEnd(0)
famafrench_data.dropna(inplace=True)
famafrench_data

/var/folders/37/zcyd0byn17lfrmxtt4_4k3s40000gn/T/ipykernel_90111/3295830964.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  famafrench_data = pd.read_csv(


,MKT-RF,SMB,HML,RMW,CMA,UMD,BAB
DATE,,,,,,,
1980-01-31,0.0550,0.0188,0.0185,-0.0184,0.0189,0.0745,0.0695
1980-02-29,-0.0123,-0.0162,0.0059,-0.0095,0.0292,0.0789,-0.0132
1980-03-31,-0.1289,-0.0697,-0.0096,0.0182,-0.0105,-0.0958,-0.1181
1980-04-30,0.0396,0.0105,0.0103,-0.0218,0.0034,-0.0048,0.0574
1980-05-31,0.0526,0.0200,0.0038,0.0043,-0.0063,-0.0118,0.0618
...,...,...,...,...,...,...,...
2025-05-31,0.0606,-0.0072,-0.0288,0.0129,0.0251,0.0221,0.0256
2025-06-30,0.0486,-0.0002,-0.0160,-0.0320,0.0145,-0.0264,0.0527
2025-07-31,0.0198,-0.0015,-0.0127,-0.0029,-0.0208,-0.0096,0.0184


## Macro + Trend

Still in Progress...

### Weights

### Backtest

### Performance Statistics

## Fundamental + Trend

In [55]:
fundamental_equity_results = pmp.run_cc_strategy_drift(
    weights      = fundamental_equity,
    returns      = equity_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [56]:
trend_equity_results = pmp.run_cc_strategy_drift(
    weights      = trend_equity,
    returns      = equity_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [57]:
trend_bond_results = pmp.run_cc_strategy_drift(
    weights      = trend_bond,
    returns      = bond_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [58]:
trend_fx_results = pmp.run_cc_strategy_drift(
    weights      = trend_fx,
    returns      = fx_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [59]:
trend_rates_results = pmp.run_cc_strategy_drift(
    weights      = trend_rates,
    returns      = rates_returns, 
    rf           = riskfree,
    frequency    = 1,
    t_cost       = 0.0002,
    benchmark    = benchmark_return
)

In [64]:
fundamental_equity_results

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_UK,w_JP,w_US
Date,,,,,,,,,,,,,
2000-11-30,0.027399,0.027399,-0.060660,0.000000,0.000000,0.0051,0.000000,0.000000,0.000000,0.000000,-0.623386,0.162907,0.179245
2000-12-31,0.018854,0.018962,0.015919,0.537498,0.000107,0.0050,0.000000,0.000000,0.000000,0.000000,-0.159189,-0.261421,0.027687
2001-01-31,-0.004059,-0.003977,0.019408,0.411558,0.000082,0.0054,0.000000,0.000000,0.000000,0.000000,0.295443,0.026827,-0.051418
2001-02-28,-0.030587,-0.030567,-0.084288,0.100037,0.000020,0.0038,0.000000,0.000000,0.000000,0.000000,0.312105,0.144955,-0.016702
2001-03-31,-0.006546,-0.006497,-0.065527,0.246314,0.000049,0.0042,0.000000,0.000000,0.000000,0.000000,0.360967,0.003137,-0.272769
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,-0.012707,-0.012629,0.043488,0.390818,0.000078,0.0034,0.090462,-0.258713,-0.644842,-0.234554,0.186717,0.061110,0.872648
2025-07-31,0.032114,0.032205,0.013121,0.458816,0.000092,0.0034,0.248686,-0.221178,-0.523279,-0.334528,0.240705,-0.321610,0.942740
2025-08-31,0.017252,0.017447,0.026408,0.974642,0.000195,0.0038,0.146615,0.093660,-0.384734,-0.226550,0.257767,0.104881,0.098647


In [62]:
import pandas as pd

# --------------------------------------------------------
# 1) List of strategy result DataFrames
# --------------------------------------------------------
dfs = [
    fundamental_equity_results,
    trend_equity_results,
    trend_bond_results,
    trend_fx_results,
    trend_rates_results,
]

# --------------------------------------------------------
# 2) Strategy weights (must sum to 1)
# --------------------------------------------------------
weights_strat = [0.50, 0.125, 0.125, 0.125, 0.125]

# --------------------------------------------------------
# 3) Align all strategies on the same date index
# --------------------------------------------------------
common_index = dfs[0].index
dfs_aligned = [df.reindex(common_index) for df in dfs]

# --------------------------------------------------------
# 4) Combine performance metrics
# --------------------------------------------------------
ret_net_combined   = sum(w * df["ret_net"]   for w, df in zip(weights_strat, dfs_aligned))
ret_gross_combined = sum(w * df["ret_gross"] for w, df in zip(weights_strat, dfs_aligned))
ret_bm_combined    = sum(w * df["ret_bm"]    for w, df in zip(weights_strat, dfs_aligned))
turnover_combined  = sum(w * df["turnover"]  for w, df in zip(weights_strat, dfs_aligned))
tcost_combined     = sum(w * df["tcost"]     for w, df in zip(weights_strat, dfs_aligned))
ret_rf_combined    = sum(w * df["ret_rf"]    for w, df in zip(weights_strat, dfs_aligned))

# --------------------------------------------------------
# 5) Create combined result table (performance only)
# --------------------------------------------------------
final_results = pd.DataFrame({
    "ret_net": ret_net_combined,
    "ret_gross": ret_gross_combined,
    "ret_bm": ret_bm_combined,
    "turnover": turnover_combined,
    "tcost": tcost_combined,
    "ret_rf": ret_rf_combined,
})

# --------------------------------------------------------
# 6) Collect all region/country weight columns across strategies
#    (some strategies may not have certain columns)
# --------------------------------------------------------
region_cols = sorted({
    col for df in dfs_aligned for col in df.columns 
    if col.startswith("w_")
})

# --------------------------------------------------------
# 7) Ensure each strategy DataFrame has all weight columns
#    Missing columns are filled with 0 (meaning: no exposure)
# --------------------------------------------------------
for i, df in enumerate(dfs_aligned):
    for reg in region_cols:
        if reg not in df.columns:
            dfs_aligned[i][reg] = 0.0

# --------------------------------------------------------
# 8) Combine weights using the strategy weights
#    This produces the final region/currency exposure
# --------------------------------------------------------
for reg in region_cols:
    final_results[reg] = sum(w * df[reg] for w, df in zip(weights_strat, dfs_aligned))

# --------------------------------------------------------
# 9) Final combined DataFrame
# --------------------------------------------------------
final_results


,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EM,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,,
2000-11-30,0.003921,0.004105,-0.060660,0.918728,0.000184,0.0051,-0.048151,0.144833,0.243112,-0.318445,-0.003637,-0.426885,0.258962
2000-12-31,0.004758,0.004948,0.015919,0.948900,0.000190,0.0050,-0.119234,0.061032,0.188992,-0.216891,-0.291526,-0.089906,0.266538
2001-01-31,0.002612,0.002809,0.019408,0.988711,0.000198,0.0054,0.272116,-0.312263,0.294825,-0.208797,0.046473,0.180525,-0.132656
2001-02-28,-0.009343,-0.009244,-0.084288,0.493037,0.000099,0.0038,0.260574,-0.193855,0.264852,-0.101982,-0.091885,0.049896,0.038620
2001-03-31,-0.002271,-0.002156,-0.065527,0.576224,0.000115,0.0042,0.210376,-0.064450,0.061171,-0.062129,-0.307101,0.195849,0.013043
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,-0.005861,-0.005682,0.043488,0.894617,0.000179,0.0034,-0.158886,0.029198,-0.085001,-0.377704,-0.100448,0.175898,0.553989
2025-07-31,0.009778,0.010084,0.013121,1.529197,0.000306,0.0034,0.080008,-0.259660,-0.391412,0.120446,-0.227117,0.304528,0.382956
2025-08-31,0.008287,0.008535,0.026408,1.239903,0.000248,0.0038,0.108985,0.074560,0.056848,-0.113868,-0.142688,0.246888,-0.185771


In [68]:
final_results.dropna(inplace=True)
pmp.run_perf_summary_benchmark_vs_strategy(final_results, alreadyXs=True)

,Benchmark,Strategy
Arithm Avg Total Return,8.4858,3.9061
Arithm Avg Xs Return,6.7769,2.1972
Std Xs Returns,15.4752,5.3087
Sharpe Arithmetic,0.4379,0.4139
Geom Avg Total Return,7.5248,3.8317
Geom Avg Xs Return,5.8039,2.1108
Sharpe Geometric,0.3750,0.3976
Min Xs Return,-19.0140,-3.4203
Max Xs Return,12.8184,6.2809
Skewness,-0.6022,0.4178
